# Equal-Weight S&P 500 Index Fund

## Library Imports

In [1]:
import pandas as pd 
import numpy as np 
import requests
import math
from api import pub

## Importing list of stocks from wikipedia

In [2]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
stocks = pd.DataFrame({'Ticker':tickers['Symbol']})
stocks

,Ticker
0,MMM
1,AOS
2,ABT
3,ABBV
4,ACN
...,...
498,YUM
499,ZBRA
500,ZBH
501,ZION


## Filling out the dataframe with API data

In [3]:
def chunks(lst, n): # converts list to list of chunks
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [4]:
# Dividing tickers by 100 to iterate batch requests
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))


my_columns = ['Ticker', 'Price', 'Market Capitalization', 'Number of Shares to Buy']
df = pd.DataFrame(columns=my_columns)


# Batch API request to IEX CLOUD
for index, symbol_string in enumerate(symbol_strings):
    batch_api_call_url = f"https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?last=2&token={pub}"
    data = requests.get(batch_api_call_url).json()
    for s_index, symbol in enumerate(symbol_string.split(',')):
        price = data[s_index]['latestPrice']
        market_cap = data[s_index]['marketCap']
        new_row = {'Ticker': symbol, 
                   "Price": price, 
                   'Market Capitalization': market_cap, 
                   'Number of Shares to Buy': 'N/A'}
        df.loc[len(df)] = new_row
df

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,MMM,109.140,60279881527,N/A
1,AOS,82.335,12578773016,N/A
2,ABT,110.440,191730304716,N/A
3,ABBV,154.885,278793000000,N/A
4,ACN,352.760,235118614731,N/A
...,...,...,...,...
498,YUM,130.940,36703558196,N/A
499,ZBRA,275.425,14145718932,N/A
500,ZBH,121.660,25424593300,N/A
501,ZION,44.440,6583732628,N/A


## Function which shows minimum budget for buying all positions in dataframe

In [5]:
def min_budget(df):
    minimum = math.ceil(np.sum(df['Price']))
    print(f'To own the above portfolio with 1 stock per company you need at least ${minimum}')
    
min_budget(df)

To own the above portfolio with 1 stock per company you need at least $100941


## Calculating the number of shares with the same monetary amount per company

In [6]:
def calc_num_of_shares(df):

    while True:
        try:
            whole_amount = int(input('Input your portfolio size in dollars -> '))
            break
        except:
            print('Do not input text, only numbers!\n')
    
    amount_on_ticker = whole_amount/len(df.index)
    df['Number of Shares to Buy'] = [math.floor(amount_on_ticker/price) for price in df['Price']]


calc_num_of_shares(df)
df

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,MMM,109.140,60279881527,1
1,AOS,82.335,12578773016,2
2,ABT,110.440,191730304716,1
3,ABBV,154.885,278793000000,1
4,ACN,352.760,235118614731,0
...,...,...,...,...
498,YUM,130.940,36703558196,1
499,ZBRA,275.425,14145718932,0
500,ZBH,121.660,25424593300,1
501,ZION,44.440,6583732628,4


## Saving dataframe to excel

In [7]:
# Create ExcelWriter object
writer = pd.ExcelWriter('data/equal_weights_S&P500.xlsx', engine='xlsxwriter')

# Write DataFrame to Excel
df.to_excel(writer, sheet_name='Recommended Trades', index=False)


background_color = '#d3d3d3' 

string_format = writer.book.add_format(
    {
        'bg_color':background_color,
        'border':1
    }
)
dollar_format_long = writer.book.add_format(
    {   
        'num_format':'$0.00',
        'bg_color':background_color,
        'border':1
    }
)
dollar_format_short = writer.book.add_format(
    {   
        'num_format':'$0',
        'bg_color':background_color,
        'border':1
    }
)
integer_format = writer.book.add_format(
    {   
        'num_format':'0',
        'bg_color':background_color,
        'border':1
    }
)

header_format = writer.book.add_format(
    {   
        'bg_color':'#008080',
        'border':1,
        'bold':True
    }
)

latin_letters_dict = {num: chr(num + 65) for num in range(26)}
col_format_dict = {
    'Ticker':string_format,
    'Price':dollar_format_long,
    'Market Capitalization':dollar_format_short,
    'Number of Shares to Buy':integer_format
}

for i, col in enumerate(df.columns):
    letter = latin_letters_dict[i]
    writer.sheets['Recommended Trades'].write(f'{letter}1', col, header_format)
    writer.sheets['Recommended Trades'].set_column(f'{letter}:{letter}', 23)
    writer.sheets['Recommended Trades'].conditional_format(1, i, len(df)+1, i,
                                                           {'type': 'no_blanks', 'format': col_format_dict[col]})

writer.close()